In [80]:
import pandas as pd
#import matplotlib.ticker as mtick
import numpy as np
import arcpy
from arcgis.gis import *
gis = GIS()

## Set Filepaths

In [81]:
# Set the workspace environment to the folder where the output shapefile will be stored
arcpy.env.workspace = 'D:/GitHub/UDOT-Master-Segments/outputs/'

# Output merged shapefile path
joinShp = 'Merged_Segments.shp'

# List of input shapefile paths
shpPaths = [
    '../data/0_USTM_v3.0 - 2023-08-17_DRAFT/Segments_UD_20230729/Segments_UD_20220729b.shp',
    '../data/2_Cache/Segments_CA_20230302/Segments_CA_20230302.shp',
    '../data/3_Dixie/Segments_DX_20220915b/Segments_DX_20220915b.shp',
    '../data/4_SuWsv2_2023-09-13_DRAFT/Segments_SW_20230913/Segments_SW_20230913.shp',
    '../data/5_IronCo - v1.0 - 2023-09-13_DRAFT/Segments_IR_20230912b/Segments_IR_20230912b.shp'
]

## Trim Segment Shapefiles

In [ ]:
# make sure each shapefile includes ONLY those segments that pertain to their subarea

## Merge Segments

In [82]:
# Use arcpy.management.Merge with FieldMappings to merge the shapefiles
fieldMappings = arcpy.FieldMappings()
fieldMappings.mergeRule = 'Join'
arcpy.management.Merge(shpPaths, joinShp, fieldMappings)

<Result 'D:/GitHub/UDOT-Master-Segments/outputs\\Merged_Segments.shp'>

## Check CRS

In [86]:
try:
    # Use arcpy.Describe to get information about the shapefile
    desc = arcpy.Describe(joinShp)
    
    # Check if the shapefile has a spatial reference (projection)
    if desc.spatialReference is not None:
        # Get the name of the coordinate system
        coordinate_system_name = desc.spatialReference.name
        
        # Print the coordinate system information
        print(f"Coordinate System: {coordinate_system_name}")
    else:
        print("The shapefile does not have a defined coordinate system.")
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

Coordinate System: NAD_1983_UTM_Zone_12N


## Check for Duplicates

In [83]:
# Create a set to store unique SEGID values
uniqueSegIDs = set()

# Create a list to store duplicate SEGID values
duplicateSegIDs = []

# Use a SearchCursor to iterate through the SEGID field
with arcpy.da.SearchCursor(joinShp, ['SEGID']) as cursor:
    for row in cursor:
        segid = row[0]
        if segid in uniqueSegIDs:
            duplicateSegIDs.append(segid)
        else:
            uniqueSegIDs.add(segid)

# Check if there are any duplicate SEGIDs
if len(duplicateSegIDs) > 0:
    print("Duplicate SEGIDs found in Merged_Segments.shp:")
    for segid in duplicateSegIDs:
        print(segid)
else:
    print("No duplicate SEGIDs found in Merged_Segments.shp.")

No duplicate SEGIDs found in Merged_Segments.shp.


## Selecting Necessary Fields

In [87]:
# Specify the input shapefile path
fields_to_keep = ["FID","SEGID", "PLANAREA", "SUBAREAID", "CO_FIPS", "DISTANCE", "FAC_WDABG","Shape"]
finalShp = r'Merged_Segments_Final.shp'

try:
    # Copy the input shapefile to the output shapefile
    arcpy.management.CopyFeatures(joinShp, finalShp)

    # Delete unwanted fields from the copied shapefile
    fields_to_delete = [field.name for field in arcpy.ListFields(finalShp) if field.name not in fields_to_keep]
    arcpy.management.DeleteField(finalShp, fields_to_delete)

    # Optionally, rename the copied shapefile to the original shapefile name if needed
    # arcpy.management.Rename(output_shapefile, input_shapefile)

    print("Columns deleted successfully.")
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

Columns deleted successfully.
